In [1]:
import tweepy
from tweepy import OAuthHandler
from pymongo import MongoClient
import time
from __future__ import unicode_literals

In [2]:
consumer_key="AzSWop5xUS0kuSx8ZcJPh3rCw"
consumer_secret="GLLJnurZnb3MSQlZmGO0uyfetyj7n0qbGdlPni3M0UPTihjscE"
access_token_key="913958769159221249-3jIFopMF4hqjm8Nn50ExcUzi2pZ0AE5"
access_token_secret="i1eHjPNrexGHOXt3H6c53EmoSLuzKHyVHwYofFZ7dGnja"
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token_key, access_token_secret)
 
api = tweepy.API(auth)

In [9]:
user_list=["justinbieber","realDonaldTrump","BarackObama","taylorswift13","cnnbrk","BillGates","espn"]

In [10]:
users=api.lookup_users(screen_names=user_list)

In [15]:
u=users[0]

In [16]:
u.id

27260086

In [17]:
userIds=list(map(lambda x:x.id,))

In [7]:
def limit_handled(cursor):
    while True:
        try:
            yield cursor.next()
        except tweepy.RateLimitError:
            time.sleep(15 * 60)


In [23]:
for page in limit_handled(tweepy.Cursor(api.followers_ids, screen_name="taylorswift13").pages(10)):
    all_follower_ids.extend(page)

E:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: generator 'limit_handled' raised StopIteration
  """Entry point for launching an IPython kernel.


In [24]:
len(set(all_follower_ids))

300717

In [20]:
len(userIds)

7

In [19]:
len(set(userIds+all_follower_ids))

300724

In [21]:
all_follower_ids=userIds+all_follower_ids

In [4]:
import json

In [5]:
with open("follower_ids.json") as f:
    all_follower_ids=json.loads(f.read())

In [22]:
with open("follower_ids.json","w") as f:
    f.write(json.dumps(all_follower_ids))

In [6]:
len(all_follower_ids)

300724

In [7]:
conn = MongoClient('127.0.0.1', 27017)
db = conn.twitter
tweets_set = db.tweets
users_set = db.users

In [44]:
tweets_set = db.tweets
users_set = db.users

In [ ]:
def get_follower_ids(screen_name,limit):
    all_follower_ids=[]
    try:
        follower_ids = followers_ids(screen_name=screen_name)
        all_follower_ids.extend(follower_ids)
    except tweepy.RateLimitError:
        time.sleep(15 * 60)
        follower_ids = followers_ids(screen_name=screen_name)
        all_follower_ids.extend(follower_ids)
    
    while len(all_follower_ids)<limit:
        try:
            follower_ids = followers_ids(screen_name=screen_name)
            all_follower_ids.extend(follower_ids)
        except tweepy.RateLimitError:
            time.sleep(15 * 60)
            continue
        if len(follower_ids)==0:
            break
    return all_follower_ids

In [8]:
def get_all_tweets(screen_name):
    try:
        tweetList = api.user_timeline(screen_name = screen_name,count=200)
        if len(tweetList)==0:
            return
        tweets_set.insert_many(list(map(lambda x:x._json,tweetList)))
    except tweepy.RateLimitError:
        time.sleep(15 * 60)
        tweetList = api.user_timeline(screen_name = screen_name,count=200)
        tweets_set.insert_many(list(map(lambda x:x._json,tweetList)))
    oldest = tweetList[-1].id - 1
    
    count=len(tweetList)
    while len(tweetList) > 0:
        try:
            tweetList = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)
            count+=len(tweetList)
#             print(time.ctime()+":"+screen_name+":"+str(count))
        except tweepy.RateLimitError:
            time.sleep(60)
            continue
        if len(tweetList)>0:
            tweets_set.insert_many(list(map(lambda x:x._json,tweetList)))
            oldest = tweetList[-1].id - 1
        else:
            break

'user.protected' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


In [10]:
batch=100
limit=100000
for i in range(8498,limit,batch):
    batch_user_list=all_follower_ids[i:i+batch]
    users = api.lookup_users(user_ids=batch_user_list)
    users_set.insert_many(list(map(lambda x:x._json,users)))
    for index,user in enumerate(users):
        if not user.protected:
            get_all_tweets(user.screen_name)
        print(time.ctime()+":"+user.screen_name+":"+str(i+index))

            


Sat Mar 31 02:41:16 2018:moussaAbdouI:8498
Sat Mar 31 02:41:16 2018:nicole80878677:8499
Sat Mar 31 02:41:16 2018:Edwin43110332:8500
Sat Mar 31 02:41:17 2018:MathapeloMoloi4:8501
Sat Mar 31 02:41:17 2018:SamukeleV:8502
Sat Mar 31 02:41:17 2018:ademildo12:8503
Sat Mar 31 02:41:28 2018:SteveOptimist:8504
Sat Mar 31 02:41:30 2018:van2jazz:8505
Sat Mar 31 02:41:31 2018:montagkristie2:8506
Sat Mar 31 02:41:31 2018:PanahiShahab:8507
Sat Mar 31 02:41:31 2018:JUNIOR53715985:8508
Sat Mar 31 02:41:31 2018:ahmed30120027:8509
Sat Mar 31 02:41:31 2018:barbpulcini:8510
Sat Mar 31 02:41:32 2018:tukrise2:8511
Sat Mar 31 02:41:32 2018:MoeEmpire1:8512
Sat Mar 31 02:41:32 2018:ttcathe:8513
Sat Mar 31 02:41:33 2018:deejfreshes:8514
Sat Mar 31 02:41:33 2018:rashid9927:8515
Sat Mar 31 02:41:33 2018:9QMklEBWsMeBOkE:8516
Sat Mar 31 02:41:34 2018:graysonspsycho:8517
Sat Mar 31 02:41:34 2018:sarahtlp:8518
Sat Mar 31 02:41:34 2018:Lahsajalah3:8519
Sat Mar 31 02:41:34 2018:cassiep0405:8520
Sat Mar 31 02:41:35 2018

Sat Mar 31 02:43:50 2018:STaFOHTtRqbcJfH:8690
Sat Mar 31 02:43:50 2018:leishakyriakod7:8691
Sat Mar 31 02:43:50 2018:malikvkshaheer:8692
Sat Mar 31 02:44:09 2018:liamendesarmy:8693
Sat Mar 31 02:44:09 2018:mohamed58964084:8694
Sat Mar 31 02:44:10 2018:shubby454:8695
Sat Mar 31 02:44:16 2018:rafaelfaisjorn:8696
Sat Mar 31 02:44:16 2018:Joazinho_27:8697
Sat Mar 31 02:44:17 2018:EmmaAnyango6:8698
Sat Mar 31 02:44:17 2018:blackwind1992:8699
Sat Mar 31 02:44:18 2018:_medoni:8700
Sat Mar 31 02:44:18 2018:SajithJohn4:8701
Sat Mar 31 02:44:18 2018:samurl00570852:8702
Sat Mar 31 02:44:19 2018:Moez91097:8703
Sat Mar 31 02:44:19 2018:Ruth95727180:8704
Sat Mar 31 02:44:19 2018:Kk7NBWP8IfoSayB:8705
Sat Mar 31 02:44:20 2018:Julio11506129:8706
Sat Mar 31 02:44:20 2018:reaswhy:8707
Sat Mar 31 02:44:23 2018:sofredora_l:8708
Sat Mar 31 02:44:23 2018:Kyriaky2:8709
Sat Mar 31 02:44:24 2018:Bod74172219:8710
Sat Mar 31 02:44:24 2018:Dudei3000:8711
Sat Mar 31 02:44:24 2018:OrhanAt73823472:8712
Sat Mar 31 02:

Sat Mar 31 02:47:11 2018:TreyFitzgerald4:8881
Sat Mar 31 02:47:12 2018:Gutierres75:8882
Sat Mar 31 02:47:12 2018:justkevinn_:8883
Sat Mar 31 02:47:12 2018:Atta_Aatir:8884
Sat Mar 31 02:47:13 2018:Celebnews1510:8885
Sat Mar 31 02:47:13 2018:Badr___97:8886
Sat Mar 31 02:47:13 2018:mashaezheva:8887
Sat Mar 31 02:47:13 2018:Cesar97232843:8888
Sat Mar 31 02:47:14 2018:egqrbn:8889
Sat Mar 31 02:47:14 2018:QkJdJO0jHa0RxG5:8890
Sat Mar 31 02:47:14 2018:Ricardo45095101:8891
Sat Mar 31 02:47:14 2018:amarhaman:8892
Sat Mar 31 02:47:17 2018:tomshuuya:8893
Sat Mar 31 02:47:18 2018:j_villamarin:8894
Sat Mar 31 02:47:18 2018:_Miirai:8895
Sat Mar 31 02:47:18 2018:ashwank811:8896
Sat Mar 31 02:47:27 2018:JimAlbin68:8897
Sat Mar 31 02:47:51 2018:mizinnie:8898
Sat Mar 31 02:47:51 2018:Krishna636263:8899
Sat Mar 31 02:47:52 2018:CsGxKBWmitRey33:8900
Sat Mar 31 02:47:52 2018:popescu36403630:8901
Sat Mar 31 02:47:52 2018:paw1028:8902
Sat Mar 31 02:47:53 2018:wazpqpazo:8903
Sat Mar 31 02:47:53 2018:mldriss19

Sat Mar 31 02:51:24 2018:krishnendude7:9071
Sat Mar 31 02:51:24 2018:nUDJNXKbw6fKhbS:9072
Sat Mar 31 02:51:25 2018:safayed3:9073
Sat Mar 31 02:51:25 2018:Kattekira:9074
Sat Mar 31 02:51:26 2018:Niko98277313:9075
Sat Mar 31 02:51:26 2018:martynakrll:9076
Sat Mar 31 02:51:26 2018:_Marrtina_:9077
Sat Mar 31 02:51:27 2018:Salih97603548:9078
Sat Mar 31 02:51:27 2018:Nykile1:9079
Sat Mar 31 02:51:27 2018:kazuko06285491:9080
Sat Mar 31 02:51:27 2018:louis12380648:9081
Sat Mar 31 02:51:27 2018:SticIvan:9082
Sat Mar 31 02:51:28 2018:gameover23345:9083
Sat Mar 31 02:51:28 2018:blurboverblurb:9084
Sat Mar 31 02:51:28 2018:RiantoHS:9085
Sat Mar 31 02:51:29 2018:Lehlogo37904238:9086
Sat Mar 31 02:51:29 2018:JoelAtkinson18:9087
Sat Mar 31 02:51:29 2018:MitchellMettle3:9088
Sat Mar 31 02:51:29 2018:Pachino861:9089
Sat Mar 31 02:51:30 2018:MaxonBaptichon:9090
Sat Mar 31 02:51:30 2018:BeratKervan3:9091
Sat Mar 31 02:51:31 2018:jpeters1973:9092
Sat Mar 31 02:51:31 2018:Amina_I01:9093
Sat Mar 31 02:51:31

Sat Mar 31 02:53:10 2018:Sahrula63288590:9261
Sat Mar 31 02:53:10 2018:sharonkern1421:9262
Sat Mar 31 02:53:18 2018:SSumpter15:9263
Sat Mar 31 02:53:18 2018:BWhiplashed49:9264
Sat Mar 31 02:53:18 2018:AndreaC34681158:9265
Sat Mar 31 02:53:18 2018:dorothy05741133:9266
Sat Mar 31 02:53:18 2018:WApdareil2001:9267
Sat Mar 31 02:53:19 2018:ngelFernandoCc2:9268
Sat Mar 31 02:53:19 2018:sekonuskahotma2:9269
Sat Mar 31 02:53:19 2018:IgnatiusSserem1:9270
Sat Mar 31 02:53:20 2018:MinnHanOfficial:9271
Sat Mar 31 02:53:20 2018:pappaws33:9272
Sat Mar 31 02:53:20 2018:UAE_BEST:9273
Sat Mar 31 02:53:21 2018:Kriskaybaby:9274
Sat Mar 31 02:53:22 2018:KevinMcCavera:9275
Sat Mar 31 02:53:22 2018:Eid62743108:9276
Sat Mar 31 02:53:22 2018:raymond_ezin:9277
Sat Mar 31 02:53:23 2018:carol16191227:9278
Sat Mar 31 02:53:23 2018:brxxwne:9279
Sat Mar 31 02:53:24 2018:NoahEri57515618:9280
Sat Mar 31 02:53:24 2018:ElvisEs01041758:9281
Sat Mar 31 02:53:24 2018:i_am_danalyn:9282
Sat Mar 31 02:53:28 2018:Lufontes157:

...

Sat Mar 31 03:45:36 2018:Vlady_Malundas:10587
Sat Mar 31 03:45:37 2018:AxstonI:10588
Sat Mar 31 03:45:37 2018:Prasenjeet134:10589
Sat Mar 31 03:45:37 2018:PiPi68583556:10590
Sat Mar 31 03:45:37 2018:Nandy10806670:10591
Sat Mar 31 03:45:38 2018:93LGP1HyUK2FGUQ:10592
Sat Mar 31 03:45:38 2018:ErmaningsihDewi:10593
Sat Mar 31 03:46:03 2018:jorgeortiznv:10594
Sat Mar 31 03:46:04 2018:Rwmurrow:10595
Sat Mar 31 03:46:04 2018:XU46GB36XE28180:10596
Sat Mar 31 03:46:04 2018:dlinh_13:10597
Sat Mar 31 03:46:05 2018:By7X3BZ7EF5GVhE:10598
Sat Mar 31 03:46:06 2018:AlexanderL_28:10599
Sat Mar 31 03:46:06 2018:zeynqt:10600
Sat Mar 31 03:46:06 2018:spxgrethan:10601
Sat Mar 31 03:46:06 2018:LaminITuray1:10602
Sat Mar 31 03:46:07 2018:YujiaFei:10603
Sat Mar 31 03:46:07 2018:nanabelleeee1:10604
Sat Mar 31 03:46:08 2018:arenas_2507:10605
Sat Mar 31 03:46:08 2018:indra22327255:10606
Sat Mar 31 03:46:09 2018:YanaCheckYouOut:10607
Sat Mar 31 03:46:09 2018:dorawendyemon:10608
Sat Mar 31 03:46:09 2018:Sarathk945

Sat Mar 31 03:57:47 2018:KamarahJackson2:10774
Sat Mar 31 03:57:48 2018:playters_lauryn:10775
Sat Mar 31 03:57:48 2018:rb_taguinod:10776
Sat Mar 31 03:57:48 2018:dhanudhajith:10777
Sat Mar 31 03:57:48 2018:northsea888:10778
Sat Mar 31 03:57:48 2018:WendellRice5:10779
Sat Mar 31 03:57:49 2018:harris_malerie:10780
Sat Mar 31 03:57:49 2018:migueltwoseven:10781
Sat Mar 31 03:57:49 2018:nic_everson:10782
Sat Mar 31 03:57:49 2018:rank_burner:10783
Sat Mar 31 03:57:50 2018:RHjbai:10784
Sat Mar 31 03:57:50 2018:VonBeGifted:10785
Sat Mar 31 03:57:52 2018:RahulRa29606806:10786
Sat Mar 31 03:57:52 2018:Adriano83680041:10787
Sat Mar 31 03:57:53 2018:LuisAlb44375980:10788
Sat Mar 31 03:57:53 2018:toby_mendenhall:10789
Sat Mar 31 03:57:53 2018:mdsaleemshaikh4:10790
Sat Mar 31 03:57:54 2018:zina18081972:10791
Sat Mar 31 03:57:54 2018:tawnya82522944:10792
Sat Mar 31 03:57:54 2018:jaybranick:10793
Sat Mar 31 03:57:54 2018:CreshingB:10794
Sat Mar 31 03:57:54 2018:MikeScheer6:10795
Sat Mar 31 03:57:55 20

Sat Mar 31 04:01:03 2018:MarcoWo64863708:10962
Sat Mar 31 04:01:04 2018:Rob518NYAudits:10963
Sat Mar 31 04:01:05 2018:Michael49042170:10964
Sat Mar 31 04:01:16 2018:ask__noriego:10965
Sat Mar 31 04:01:16 2018:TemplaLarry:10966
Sat Mar 31 04:01:16 2018:Mohamed90012972:10967
Sat Mar 31 04:01:16 2018:izzatzahri25:10968
Sat Mar 31 04:01:17 2018:LeeZhiXian2:10969
Sat Mar 31 04:01:17 2018:ruus84266088:10970
Sat Mar 31 04:01:17 2018:Rena_Ryugu220:10971
Sat Mar 31 04:01:18 2018:EthanReeves33:10972
Sat Mar 31 04:01:18 2018:sugar_bones_:10973
Sat Mar 31 04:01:19 2018:mianarslanjani1:10974
Sat Mar 31 04:01:19 2018:SARMADBEZZO89:10975
Sat Mar 31 04:01:19 2018:gwenolantern:10976
Sat Mar 31 04:01:19 2018:Novlerios:10977
Sat Mar 31 04:01:19 2018:DairoQuintero9:10978
Sat Mar 31 04:01:19 2018:SridharUsh1:10979
Sat Mar 31 04:01:19 2018:Fernand53419085:10980
Sat Mar 31 04:01:20 2018:Kamilit08244728:10981
Sat Mar 31 04:01:20 2018:niheqcywv8717sb:10982
Sat Mar 31 04:01:20 2018:Brbara59302034:10983
Sat Mar 

Sat Mar 31 04:19:37 2018:JGfhMtkxJ3hSaad:11148
Sat Mar 31 04:19:37 2018:dianegabriele15:11149
Sat Mar 31 04:19:38 2018:ErdemSx:11150
Sat Mar 31 04:19:38 2018:JoseMan32099138:11151
Sat Mar 31 04:19:39 2018:BuddyGamesMovie:11152
Sat Mar 31 04:19:39 2018:tammyblunt7:11153
Sat Mar 31 04:19:39 2018:agueroeduardof1:11154
Sat Mar 31 04:19:39 2018:boypanuwun:11155
Sat Mar 31 04:19:39 2018:EmmaWagner888:11156


TweepError: [{'message': 'Over capacity', 'code': 130}]

In [43]:
for u in user:
    print(u.screen_name)

justinbieber
realDonaldTrump


In [15]:
",".join(user_list)

'justinbieber,realDonaldTrump'

In [42]:
a[2].urls

[URL(URL=https://t.co/53KZ5720Tw, ExpandedURL=https://twitter.com/mileycyrus/status/974510172042289153)]

In [44]:
t=a[1]

In [48]:
t.text

'Proud of these kids https://t.co/M796VNgWWV'

[Media(ID=976131076501291008, Type=photo, DisplayURL='pic.twitter.com/M796VNgWWV')]

In [ ]:
api.GetFollowerIDs(screen_name=user,total_count=u.followers_count)#too many followers will make this slow

In [ ]:
for user in user_list:
    u=api.GetUser(screen_name=user)
    """u.followers_count
    u.friends_count
    u.created_at
    u.listed_count
    u.statuses_count
    follower_ids=api.get"""
    get_tweets(user)
    

In [ ]:
def get_tweets(user):
    tweets=api.GetUserTimeline(screen_name=user,count=201)
    len(t.media) # know whether this is a pure text tweet